Les imports

In [3]:
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

from keras.preprocessing.image import ImageDataGenerator

import math
import time

from commun.CommunFonction import *
from commun.CONSTANTE import *
from commun.datasets.dataManager import *

Paramètres Généraux + activation du temps

In [4]:
beginTime = time.time()
dir = "../commun/datasets/CNN"
data = DataManager(dir)


Construction de l'IA

In [5]:
def buildCNN():
    model = Sequential()
    model.add(Convolution2D(filters=NUMBER_FILTRE, kernel_size=KERNEL_SIZE, strides=STRIDE_CONVOLUTION, input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3),
                            activation=FUNCTION_ACTIVATION))

    model.add(MaxPooling2D(pool_size=SIZE_MATRICE_POOLING, strides=STRIDE_POOLING))

    nbrLayer = 1
    multiplicateur = 1
    for i in range(NUMBER_LAYER_POOL_CONV - 1):
        model.add(Convolution2D(filters=NUMBER_FILTRE * multiplicateur, kernel_size=KERNEL_SIZE, strides=STRIDE_CONVOLUTION,
                                activation=FUNCTION_ACTIVATION))

        model.add(MaxPooling2D(pool_size=SIZE_MATRICE_POOLING, strides=STRIDE_POOLING))
        nbrLayer += 1
        multiplicateur = math.floor((nbrLayer / 2) + 1)

    model.add(Flatten())

    for j in range(NUMBER_HIDDEN_LAYER):
        model.add(Dense(units=NUMBER_NEURONE_HIDDEN_LAYER_CNN, activation=FUNCTION_ACTIVATION))

    model.add(Dense(units=NUMBER_NEURONE_OUTPUT, activation=FUNCTION_OUTPOUT))

    model.compile(optimizer=OPTIMIZER, loss=FUNCTION_LOSS, metrics=METRIC)

    return model



CNN = buildCNN()

Initialisation des jeux des données

In [6]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train = train_datagen.flow_from_directory(data.getPathTrainPicture(),
                                          target_size=IMAGE_SIZE_TUPLE,
                                          batch_size=BATCH_SIZE,
                                          class_mode=CLASS_MODE)

test = test_datagen.flow_from_directory(data.getPathTestPicture(),
                                          target_size=IMAGE_SIZE_TUPLE,
                                          batch_size=BATCH_SIZE,
                                          class_mode=CLASS_MODE)

Found 8005 images belonging to 2 classes.
Found 2023 images belonging to 2 classes.


Entrainement de l'IA + fin temps

In [7]:
saves = CNN.fit(
    train,
    steps_per_epoch=math.ceil(data.getNumberTrain() / BATCH_SIZE),
    epochs=NUMBER_EPOCH,
    validation_data=test,
    validation_steps=math.ceil(data.getNumberTest() / BATCH_SIZE),
    verbose=1)

endTime = time.time() - beginTime

Epoch 1/15
 43/251 [====>.........................] - ETA: 52s - loss: 0.6907 - acc: 0.5315

KeyboardInterrupt: 

Sauvegarde des Information lier a l'apprentissage

In [ ]:
writeIntoFilesKeras(saves,endTime,"keras_CNN")